In [15]:
import pandas as pd

# Collect Death Rate Data
death_rate = pd.read_csv('../deaths/death-rates.csv')

# Collect Population Data
population = pd.read_csv('../census/data/table1_zcta.csv')
population = population[['Zip Code', 'Population']]

# Collect Income Data
income = pd.read_csv('../income/data/returns-aig.csv')[::7]
income = income[['Zip Code', 'Returns', 'AIG']].reset_index()

# Collect total number of pills data.
pills = pd.read_csv('../arcos-ca/pills-per-zipcode.csv')

# Begin merging all data sets into a single dataframe.
# They are sorted by Pills columns.
merge = pd.merge(pills, income, how='left', on='Zip Code')
del merge['index']
merge = pd.merge(merge, population, how='left', on='Zip Code')
merge = pd.merge(merge, death_rate, how='left', on='Zip Code')

# Remove Missing Value rows and Zero Pill value rows.
merge = merge.dropna()
merge = merge[merge['Pills'] > 0]
merge = merge[merge['Death Rate'] > 0]

# Revome commas and convert object columns to numeric columns.
merge['Returns'] = merge['Returns'].str.replace(',', '').astype('int64')
merge['AIG'] = merge['AIG'].str.replace(',', '').astype('int64')
merge['Population'] = merge['Population'].astype('int64')

# Rename mispelled columns names and be more descriptive.
merge = merge.rename(columns={'Returns': 'Tax Returns', 'AIG': 'AGI thousands'})
merge = merge.reset_index(drop=True)
merge

,Zip Code,City,Pills,Tax Returns,AGI thousands,Population,Death Rate
0,94550,LIVERMORE,52022690,21341,2066480,46896,1.63
1,90242,DOWNEY,21791650,17767,726095,43497,2.65
2,92010,CARLSBAD,10548200,6737,495599,14382,12.55
3,95350,MODESTO,6547370,20938,891634,51046,2.19
4,95825,SACRAMENTO,6087880,13817,591794,31084,2.75
...,...,...,...,...,...,...,...
668,95033,LOS GATOS,200,4012,552488,9372,5.35
669,93725,FRESNO,180,9020,280513,24979,4.43
670,95215,STOCKTON,130,8467,316574,23224,35.97
671,94930,FAIRFAX,100,4029,319932,8500,5.44


In [16]:
merge.to_csv('data/merged.csv', index=False)